<a href="https://colab.research.google.com/github/koki0702/zerobook3/blob/master/notebook/ja/step09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**前ステップまでに実装したコード**

In [1]:
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

***

# ステップ9 関数をより便利に

私たちのDeZeroは、今ではバックプロパゲーションで計算ができます。さらには、Define-by-Runという、実行時に計算の「つながり」を作る特徴を持ち合わせています。ここでは、今のDeZeroをより使いやすくするために、DeZeroの関数に対して3つの改善を行います。

## 9.1  Pythonの関数として利用

これまで私たちは、DeZeroで使用する関数を「Pythonのクラス」として実装してき

In [2]:
x = Variable(np.array(0.5))
f = Square()
y = f(x)

上記のように、2乗の計算を行うには、`Square`クラスのインスタンスを生成し、そのインスタンスを呼び出すという2段階のステップを踏みます。しかし、使う側からしてみれば、この2段階の作業は少し手間です（`y = Square()(x)`とも書けますが、それも不格好です）。より好ましいのは、「Pythonの関数」として利用できることでしょう。そこで、次の実装を加えます。

In [3]:
def square(x):
    f = Square()
    return f(x)

def exp(x):
    f = Exp()
    return f(x)

上記のように、`square`と`exp`という2つの関数を実装しました。これで「DeZeroの関数」を「Pythonの関数」として利用できるようになります。ちなみに上のコードは、次のように1行で書くこともできます。

In [4]:
def square(x):
    return Square()(x)  # 1行でまとめて書く

def exp(x):
    return Exp()(x)

`f = Square()`のように`f`という変数名で参照することなく、直接`Square()(x)`と書くこともできます。それでは、ここで実装した2つの関数を使ってみましょう。

In [5]:
x = Variable(np.array(0.5))
a = square(x)
b = exp(a)
y = square(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


このとおり、最初の`np.array(0.5)`を`Variable`で包めば、通常の数値計算を行うような感覚で――NumPyを使って計算するように――コーディングできます。なお、上のコードは、関数を連続して適用することも可能です。その場合は、次のように書くことができます。

In [6]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))  # ★連続して適用
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


これで、より自然なコードで計算ができるようになりました。これが1つ目の改善点です。

## 9.2 backwardメソッドの簡略化

2つ目の改善点は、逆伝播におけるユーザの手間を減らすためのものです。具体的には、先ほど書いたコードにある`y.grad = np.array(1.0)`を省略します。というのも、私たちは逆伝播を行う際に、毎回`y.grad = np.array(1.0)`というコードを書いています。その作業を省略できるように、`Variable`の`backward`メソッドの中に次の2行を追加します。

In [7]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:  # ★
            self.grad = np.ones_like(self.data)  # ★

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator) 

上記のように、もし変数の`grad`が`None`の場合、自動で微分を生成します。ここでは、`np.ones_like(self.data)`によって、`self.data`と同じ形状かつ同じデータ型で、その要素が1の`ndarray`インスタンスを生成します。`self.data`がスカラの場合は、`self.grad`もスカラになります。

<br><div class="alert alert-info">

<b>NOTE</b>

これまでは出力の微分を<code>np.array(1.0)</code>としていましたが、上のコードでは<code>np.ones_like()</code>を使いました。その理由は、<code>Variable</code>の<code>data</code>と<code>grad</code>のデータ型を同じにするためです。たとえば、<code>data</code>の型が32ビットの浮動小数点数であれば、<code>grad</code>の型も32ビットの浮動小数点数になります。ちなみに、<code>np.array(1.0)</code>と書いた場合、そのデータ型は64ビットの浮動小数点数になります。

</div><br>

これで、何らかの計算を行えば、後は最終出力の変数に対して`backward`メソッドを呼ぶだけで微分が求まります。実際に試してみると、次のようになります。


In [8]:
x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256
